In [1]:
from datetime import datetime as dt
import pandas as pd
import time
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')    
    return df
    
df = extract_data(url_by_city)
df.head()


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-09-01,2017-09-30,30,place,6,29470,f,"Chicago, IL",Chicago,Illinois,IL,Multi-Family (2-4 Unit),4,259500.0,0.017647,0.166292,285000.0,0.036364,0.017893,107.175090,0.053260,0.265550,207.142857,0.036060,0.002862,284.0,-0.230352,-0.089744,421.0,-0.060268,-0.029954,476.0,-0.044177,-0.004184,1316.0,0.021739,-0.147116,4.6,1.1,-0.3,42.0,6.0,-2.0,0.977178,0.001736,-0.000593,0.271127,-0.037816,-0.001309,0.266717,-0.021326,-0.010664,0.285036,-0.063179,0.047708,"Chicago, IL",16984,2024-09-18 14:24:31
1,2023-03-01,2023-03-31,30,place,6,38334,f,"Lexington, WA",Lexington,Washington,WA,All Residential,-1,424900.0,0.183565,0.103636,429900.0,NaN,0.116623,210.763889,-0.151149,-0.178842,251.454522,NaN,0.080750,3.0,-0.250000,0.000000,2.0,-0.333333,-0.714286,2.0,NaN,-0.777778,4.0,0.000000,0.000000,1.3,0.3,0.0,44.0,29.0,38.0,0.973666,-0.038899,-0.083965,0.333333,-0.416667,-0.333333,0.250000,-0.500000,NaN,0.500000,0.500000,-0.214286,"Longview, WA",31020,2024-09-18 14:24:31
2,2020-07-01,2020-07-31,30,place,6,37598,f,"Parsippany, NJ",Parsippany,New Jersey,NJ,All Residential,-1,485000.0,-0.067308,-0.035180,477500.0,-0.005208,0.063474,236.218595,-0.004243,-0.094639,242.123687,-0.012999,-0.017559,14.0,0.750000,-0.222222,15.0,0.363636,-0.285714,18.0,-0.379310,0.636364,20.0,-0.285714,-0.310345,1.4,-2.1,-0.2,33.0,3.0,16.0,0.982790,-0.007551,-0.024976,0.357143,0.107143,-0.198413,0.250000,0.142857,0.043103,0.000000,-0.090909,-0.047619,"Newark, NJ",35084,2024-09-18 14:24:31
3,2022-09-01,2022-09-30,30,place,6,14794,f,"Osceola, IN",Osceola,Indiana,IN,All Residential,-1,295050.0,-0.242175,0.076825,299000.0,0.071685,0.272882,131.236489,-0.272939,-0.018186,139.920000,0.190650,-0.125188,6.0,-0.250000,0.500000,8.0,-0.111111,1.000000,5.0,-0.285714,0.000000,11.0,0.100000,-0.266667,1.8,0.5,-2.0,17.0,-32.0,-68.0,0.997001,-0.005324,0.008542,0.333333,-0.166667,0.083333,0.272727,-0.127273,NaN,0.375000,0.152778,0.375000,"South Bend, IN",43780,2024-09-18 14:24:31
4,2019-06-01,2019-06-30,30,place,6,16196,f,"Southern Pines, NC",Southern Pines,North Carolina,NC,All Residential,-1,316500.0,-0.055224,0.049751,314000.0,-0.083212,-0.033846,149.049656,0.055943,0.118620,141.572238,0.048967,0.021164,36.0,-0.389831,-0.379310,35.0,-0.406780,-0.375000,30.0,-0.210526,0.200000,171.0,-0.055249,-0.246696,4.8,1.7,0.9,96.0,18.0,-20.0,0.975310,-0.003670,0.005111,0.111111,0.060264,-0.009579,NaN,NaN,NaN,0.000000,-0.016949,-0.017857,"Pinehurst, NC",38240,2024-09-18 14:24:31


In [3]:
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))

Num of rows: 5559476
Num of cols: 58


In [4]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def lambda_handler(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')
   
   
     # Remove commas from the 'city' column
    df['city'] = df['city'].str.replace(',', '')
    cols = ['period_begin','period_end','period_duration', 'region_type', 'region_type_id', 'table_id',
    'is_seasonally_adjusted', 'city', 'state', 'state_code', 'property_type', 'property_type_id',
    'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold',
    'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'parent_metro_region_metro_code', 
    'last_updated']
    df = df[cols]
    
    df = df.dropna()
    
    #let's change the period_begin and period_end to date time object and extract years and month.
    df['period_begin'] = pd.to_datetime(df['period_begin'])
    df['period_end'] = pd.to_datetime(df['period_end'])
    
    df["period_begin_in_years"] = df['period_begin'].dt.year
    df["period_end_in_years"] = df['period_end'].dt.year

    df["period_begin_in_months"] = df['period_begin'].dt.month
    df["period_end_in_months"] = df['period_end'].dt.month
    
    #let's map the month number to their respective month name.
    month_dict = {
        "period_begin_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec",
        },
        "period_end_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec"
        }}
    df = df.replace(month_dict)
    print('Num of rows:', len(df))
    print('Num of cols:', len(df.columns))
    return df

In [5]:
df = lambda_handler(url_by_city)
df.head()

Num of rows: 4331055
Num of cols: 28


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,city,state,state_code,property_type,property_type_id,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,parent_metro_region_metro_code,last_updated,period_begin_in_years,period_end_in_years,period_begin_in_months,period_end_in_months
0,2017-09-01,2017-09-30,30,place,6,29470,f,Chicago,Illinois,IL,Multi-Family (2-4 Unit),4,259500.0,285000.0,107.175090,207.142857,284.0,1316.0,4.6,42.0,0.977178,0.271127,16984,2024-09-18 14:24:31,2017,2017,Sep,Sep
1,2023-03-01,2023-03-31,30,place,6,38334,f,Lexington,Washington,WA,All Residential,-1,424900.0,429900.0,210.763889,251.454522,3.0,4.0,1.3,44.0,0.973666,0.333333,31020,2024-09-18 14:24:31,2023,2023,Mar,Mar
2,2020-07-01,2020-07-31,30,place,6,37598,f,Parsippany,New Jersey,NJ,All Residential,-1,485000.0,477500.0,236.218595,242.123687,14.0,20.0,1.4,33.0,0.982790,0.357143,35084,2024-09-18 14:24:31,2020,2020,Jul,Jul
3,2022-09-01,2022-09-30,30,place,6,14794,f,Osceola,Indiana,IN,All Residential,-1,295050.0,299000.0,131.236489,139.920000,6.0,11.0,1.8,17.0,0.997001,0.333333,43780,2024-09-18 14:24:31,2022,2022,Sep,Sep
4,2019-06-01,2019-06-30,30,place,6,16196,f,Southern Pines,North Carolina,NC,All Residential,-1,316500.0,314000.0,149.049656,141.572238,36.0,171.0,4.8,96.0,0.975310,0.111111,38240,2024-09-18 14:24:31,2019,2019,Jun,Jun


In [6]:
#number of years in our dataset
sorted(df["period_begin_in_years"].unique())

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

In [ ]:
# download file
# df.to_csv('real_estate_w209_byCity.csv', index=False)